<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/Refactoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import download
ss = SnowballStemmer('english')
from spacy import load
from spacy.matcher import Matcher
from tqdm import tqdm
nlp = load("en_core_web_sm", exclude=["tok2vec", "tagger", "parser",'senter', "attribute_ruler", "lemmatizer", 'ner'])
# from google.colab import drive
# drive.mount('/content/drive/')
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
download('punkt')

[nltk_data] Downloading package punkt to /Users/francisco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
mp.cpu_count()

8

In [15]:
directory = '/content/drive/MyDrive/GHDomains/Downloaded_repos/'
local_directory = '/Volumes/GoogleDrive/My Drive/GHDomains/Downloaded_repos/'
directory = local_directory

In [16]:
word_list = ['refactor', 'restruct', 'clean', 'not used', 'unused', 'reformat', 'import', 'remove', 'replace', 'split', 'reorg', 'rename', 'move']

matcher = Matcher(nlp.vocab)

# Add patterns
patterns= [[{"LOWER": "refactor"}],
            [{"LOWER": "restruct"}],
            [{"LOWER": "clean"}],
            [{"LOWER": "not"},{"LOWER": "use"}],    # used      steam = use
            [{"LOWER": "unus"}],                    # unused    steam = unus
            [{"LOWER": "reformat"}],                ## steam = reform
            [{"LOWER": "import"}],
            [{"LOWER": "remov"}],                   # remove    steam = remov
            [{"LOWER": "replac"}],                  # replace   steam = replac
            [{"LOWER": "split"}],                   ## split     steam = splite / split
            [{"LOWER": "reorg"}],                   # reorg     steam = reorg
            [{"LOWER": "renam"}],                   # rename    steam = renam
            [{"LOWER": "move"}]]

matcher.add("refactoring_pattern", patterns)

In [17]:
def word_matcher(text):
    
    try:
        with nlp.disable_pipes():
        # Tokenize the commit sentence:
            text = word_tokenize(text)
        # apply steam technique
            text = ' '.join([ss.stem(word) for word in text])
        # select matching words
            text = nlp(text)
            words = matcher(text)
    except TypeError:
        return 0

    if len(words)>0:

        return 1

    else:

        return 0

In [18]:
repos_ids = [file_name for file_name in os.listdir(directory) if file_name.endswith('.json') and not file_name.startswith('all_repos')]
repos_ids = list(set(repos_ids))
len(repos_ids)

889

In [19]:
matches = []
to_rescan = []

for filename in tqdm(repos_ids):
      try:
          aux_reader = pd.read_json(directory+filename)
          commits    = aux_reader['Messages:'].size
          match      = aux_reader['Messages:'].apply(word_matcher).sum()
          filename   = filename.replace(':', '/').replace('.json', '')
          matches.extend([(filename, commits, match)])
      except:
          print('None commits found in: ', filename)
          filename   = filename.replace(':', '/')
          filename   = filename.replace('.json', '')
          to_rescan.extend([filename])


df = pd.DataFrame(matches, columns =['Name', 'Commits', 'Matches'])
df

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 889/889 [19:00<00:00,  1.28s/it]


,Name,Commits,Matches
0,Netflix/pollyjs,364,54
1,dotnet/runtime,116316,21904
2,plouc/nivo,1745,173
3,github/copilot-docs,22,3
4,jamiebuilds/babel-handbook,281,11
...,...,...,...
884,QSCTech/zju-icicles,419,8
885,phanan/htaccess,156,19
886,ClickHouse/ClickHouse,84357,6875
887,facebookarchive/caffe2,3680,644


In [20]:
df_path = '/content/drive/MyDrive/GHDomains/Resources/new_popular_df_predicted.csv'
df_path = '/Volumes/GoogleDrive/My Drive/GHDomains/Resources/new_popular_df_predicted.csv'

new_popular_df = pd.read_csv(df_path, usecols=['Name','Predicted_labels'])
new_popular_df

,Name,Predicted_labels
0,vuejs/devtools,Web libraries and frameworks
1,futurice/android-best-practices,Documentation
2,microsoft/Web-Dev-For-Beginners,Documentation
3,airbnb/react-sketchapp,Web libraries and frameworks
4,eugeneyan/applied-ml,Documentation
...,...,...
888,graphql/dataloader,Non-web libraries and frameworks
889,junyanz/CycleGAN,Non-web libraries and frameworks
890,Tencent/wcdb,Application & System software
891,halfrost/Halfrost-Field,Non-web libraries and frameworks


In [21]:
join_data = pd.merge(new_popular_df, df, on ='Name', how ='inner')
join_data['Non refactoring commits'] = join_data['Commits'] - join_data['Matches']

columns = ['Name','Domain','Total commits', 'Refactoring commits', 'Non refactoring commits']
join_data.columns = columns
join_data = join_data[['Name','Domain', 'Refactoring commits', 'Non refactoring commits','Total commits']]
join_data

,Name,Domain,Refactoring commits,Non refactoring commits,Total commits
0,vuejs/devtools,Web libraries and frameworks,246,1723,1969
1,futurice/android-best-practices,Documentation,31,216,247
2,microsoft/Web-Dev-For-Beginners,Documentation,60,1205,1265
3,airbnb/react-sketchapp,Web libraries and frameworks,123,657,780
4,eugeneyan/applied-ml,Documentation,22,377,399
...,...,...,...,...,...
884,graphql/dataloader,Non-web libraries and frameworks,13,156,169
885,junyanz/CycleGAN,Non-web libraries and frameworks,6,92,98
886,Tencent/wcdb,Application & System software,212,858,1070
887,halfrost/Halfrost-Field,Non-web libraries and frameworks,1,1025,1026


In [22]:
df = join_data.groupby(['Domain']).sum()
df

,Refactoring commits,Non refactoring commits,Total commits
Domain,,,
Application & System software,99034,617733,716767
Documentation,24753,386479,411232
Non-web libraries and frameworks,76829,430573,507402
Software tools,138194,913870,1052064
Web libraries and frameworks,59108,418586,477694


In [23]:
from math import sqrt 

def phi_effect_size(contingency_table):
  # interpretation for degree of freedom = 1
  # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5426219/#B2
  effect_size = sqrt(stat/sum(sum(contingency_table)))

  if effect_size <= 0.10:
    interpretation = 'small'
  elif 0.10 < effect_size <= 0.30:
    interpretation = 'small to medium'
  elif 0.30 < effect_size <= 0.50:
    interpretation = 'medium to large'
  elif effect_size > 0.50:
    interpretation = 'large'

  return round(effect_size,3), interpretation

In [24]:
domains = join_data['Domain'].unique()

for domain in domains:
    print(domain, ':')
    data1 = df.loc[domain]
    data2 = df.loc[domains[domains != domain]].sum()
    aux = np.column_stack((data1, data2))[:2].T
    print(aux)
    stat, p, dof, expected = chi2_contingency(aux)
    print('degrees of freedom = %d' % dof)
    print(expected)
    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    print('Effect size: ', phi_effect_size(aux))
    print('=========================================')

Web libraries and frameworks :
[[  59108  418586]
 [ 338810 2348655]]
degrees of freedom = 1
[[  60054.81591667  417639.18408333]
 [ 337863.18408333 2349601.81591667]]
probability=0.950, critical=3.841, stat=20.087
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
Effect size:  (0.003, 'small')
Documentation :
[[  24753  386479]
 [ 373165 2380762]]
degrees of freedom = 1
[[  51699.33484416  359532.66515584]
 [ 346218.66515584 2407708.33484416]]
probability=0.950, critical=3.841, stat=18462.478
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
Effect size:  (0.076, 'small')
Software tools :
[[ 138194  913870]
 [ 259724 1853371]]
degrees of freedom = 1
[[ 132263.5617206  919800.4382794]
 [ 265654.4382794 1847440.5617206]]
probability=0.950, critical=3.841, stat=455.497
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
Effect size:  (0.012, 'small')
Application & System software :
[[  99034  617733]
 [ 298884 2149508]]
degr